In [1]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer, util
from InstructorEmbedding import INSTRUCTOR


In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
#Bert Models
from sentence_transformers import SentenceTransformer,util
tmodel = SentenceTransformer('stsb-roberta-large')
tmodel2 = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

KeyboardInterrupt: 